In [123]:
import requests, os
import json
from hdfs import InsecureClient
import getpass
import fsspec

In [2]:
save_dir = r"C:\Users\user\Documents\python\Samsung\Data"
os.makedirs(save_dir, exist_ok=True)

# Crawl news

In [60]:
def save_data_to_json(data, filename):
    filepath = os.path.join(save_dir, filename)
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as file:
            all_data = json.load(file)
    else:
        all_data = []
        
    all_data.extend(data)
    
    with open(filepath, 'w', encoding='utf-8') as file:
        json.dump(all_data, file, ensure_ascii=False, indent=2)

## Tin ngành

In [61]:
def industry_news(company, page=1, page_size=10):
  url = "https://finance.vietstock.vn/data/getnewsbychannel3"

  payload = f"code={company}&type=1&page={page}&pageSize={page_size}"
  headers = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Origin': 'https://finance.vietstock.vn',
    # 'Referer': 'https://finance.vietstock.vn/HPG/tin-tuc-su-kien.htm',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  # Nếu status không phải 200 thì coi như hết data
  if response.status_code != 200 or not response.text.strip():
      return []
  # Thử parse JSON, nếu lỗi thì trả về list rỗng
  try:
      return json.loads(response.text)
  except json.JSONDecodeError:
      return []


## KQKD - cổ tức - tăng vốn - giao dịch cổ đông - thay đổi nhân cự

In [62]:
def company_news(company, page=1, page_size=5):
  url = "https://finance.vietstock.vn/data/getnewsbycode"

  payload = f"code={company}&types%5B%5D=-1&types%5B%5D=3&types%5B%5D=4&types%5B%5D=5&types%5B%5D=6&types%5B%5D=7&page={page}&pageSize={page_size}&__RequestVerificationToken=FMoHc09wZmtJCVRbYqogIpFeHSqCBGlnUHVVxVmt_R-GnUmlnLo_xejjyJUwmeChhmu4ocSeRzZzmGsIRvDWGeElWt18ZMqXIu50_sw0FGM1"
  headers = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Cookie': 'language=vi-VN; Theme=Light; AnonymousNotification=; ASP.NET_SessionId=wzregf3xrfbncs5ijkhafdly; __RequestVerificationToken=PN_MuSdY1zIn2HFNJ_Fi2waRhEaEFuRlbG2JIBHK50FKQLMX7dXlN0lHUw7oKwtptNp1vPD0z6fqp7sjeZ4Ss8ATNLG4gh2N8DoA5K2DQ7M1; finance_viewedstock=HPG,'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  # Nếu status không phải 200 thì coi như hết data
  if response.status_code != 200 or not response.text.strip():
      return []
  # Thử parse JSON, nếu lỗi thì trả về list rỗng
  try:
      return json.loads(response.text)
  except json.JSONDecodeError:
      return []

## Lịch sự kiện của HPG

### Cổ tức tiền - Cổ tức cổ phiếu - Thưởng - Phát hành thêm
### Niêm yết
### Đại hội cổ đông

In [46]:
def events_news(company, typeID=1, page=1, page_size=5):
    url = "https://finance.vietstock.vn/data/eventstypedata"

    payload = f"eventTypeID={typeID}&channelID=0&code={company}&catID=-1&fDate=&tDate=&page={page}&pageSize={page_size}&orderBy=Date1&orderDir=DESC&__RequestVerificationToken=4haikoHUvxjcsxFBoT0MiTOxzZIzVW39tG6jAvomkXpnkCX2eHste_nTh_Cla0k_kpdz1s2DVTHP8zLvWxnOzLBKXXcxGcnwe-549S7-r0k1"
    headers = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Cookie': 'language=vi-VN; Theme=Light; AnonymousNotification=; ASP.NET_SessionId=f2j1rp1fbzrmmoopiukvcjdx; __RequestVerificationToken=zKJasuy6Ku14toGHvABpdJIEtapJHgcfWZQCCJ6xA-AbHMwPMw0gD_Jueb3LDrtIp1q-LHqBF02yjADYwulu5CESidHeSKteLoXAZGIXPhQ1; finance_viewedstock=HPG,'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    # Nếu status không phải 200 thì coi như hết data
    if response.status_code != 200 or not response.text.strip():
        return []
    # Thử parse JSON, nếu lỗi thì trả về list rỗng
    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return []
    
    # return json.loads(response.text)


### Doanh nghiệp cùng ngành

In [48]:
def same_industry_events_news(company, typeID=1, page=1, page_size=5):
  url = "https://finance.vietstock.vn/data/eventstypedatasameindustry"

  payload = f"eventTypeID={typeID}&channelID=0&code={company}&catID=-1&fDate=&tDate=&page={page}&pageSize={page_size}&orderBy=Date1&orderDir=DESC&__RequestVerificationToken=4haikoHUvxjcsxFBoT0MiTOxzZIzVW39tG6jAvomkXpnkCX2eHste_nTh_Cla0k_kpdz1s2DVTHP8zLvWxnOzLBKXXcxGcnwe-549S7-r0k1"
  headers = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Cookie': 'language=vi-VN; Theme=Light; AnonymousNotification=; ASP.NET_SessionId=f2j1rp1fbzrmmoopiukvcjdx; __RequestVerificationToken=zKJasuy6Ku14toGHvABpdJIEtapJHgcfWZQCCJ6xA-AbHMwPMw0gD_Jueb3LDrtIp1q-LHqBF02yjADYwulu5CESidHeSKteLoXAZGIXPhQ1; finance_viewedstock=HPG,'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  # Nếu status không phải 200 thì coi như hết data
  if response.status_code != 200 or not response.text.strip():
      return []
  # Thử parse JSON, nếu lỗi thì trả về list rỗng
  try:
      return json.loads(response.text)
  except json.JSONDecodeError:
      return []


## Start Crawling

In [ ]:
# # 1) Định nghĩa list các công ty
# default_companies = ['HPG', 'HSG']

# # 2) Định nghĩa các “task” cần crawl
# tasks = [
#     # industry_news chỉ theo page, lưu chung file
#     {
#         'func': industry_news,
#         'fixed_args': [],            # không có tham số thêm ngoài company & page
#         'filename_tpl': '{company}_industry_news.json',
#         'max_pages': 200
#     },
#     # company_news có 5 loại nội dung, index 1..5 → mỗi loại lưu file riêng
#     {
#         'func': company_news,
#         'fixed_args': [],            # company & page, page_size mặc định
#         'is_categorized': True,
#         'category_map': {
#             1: '{company}_Results_Business_Plan.json',
#             2: '{company}_Dividend.json',
#             3: '{company}_Capital_Increase_M&A.json',
#             4: '{company}_Shareholder_transactions.json',
#             5: '{company}_Personnel_changes.json'
#         },
#         'max_pages': 25
#     },
#     # events_news có typeID 1,2,5
#     {
#         'func': events_news,
#         'fixed_args': [],            # company
#         'is_categorized': True,
#         'category_map': {
#             1: '{company}_event_news.json',
#             2: '{company}_listing_news.json',
#             5: '{company}_Shareholders_meeting_news.json'
#         },
#         'max_pages': 10
#     },
#     # same_industry_events_news tương tự
#     {
#         'func': same_industry_events_news,
#         'fixed_args': [],
#         'is_categorized': True,
#         'category_map': {
#             1: 'same_industry_raw_materials_event_news.json',
#             2: 'same_industry_raw_materials_listing_news.json',
#             5: 'same_industry_raw_materials_Shareholders_meeting_news.json'
#         },
#         'max_pages': 200,
#         'companies': ['HPG']  # Chỉ lấy Hòa Phát, vì công ty cùng ngành đều có dữ liệu giống nhau
#     }
# ]

# # Runner chung
# for task in tasks:
#     # lấy danh sách company cho task này (mặc định hoặc override)
#     companies = task.get('companies', default_companies)

#     for comp in companies:
#         # nếu task không phân loại theo typeID/loại
#         if not task.get('is_categorized', False):
#             for page in range(1, task['max_pages'] + 1):
#                 data = task['func'](comp, page=page)
#                 if not data:
#                     break
#                 fn = task['filename_tpl'].format(company=comp)
#                 save_data_to_json(data, filename=fn)

#         # nếu có phân loại theo typeID/loại
#         else:
#             for category, filename_tpl in task['category_map'].items():
#                 for page in range(1, task['max_pages']):
#                     # gọi hàm với thêm param category (typeID hoặc index)
#                     data = task['func'](comp, typeID=category, page=page)
#                     if not data:
#                         break
#                     fn = filename_tpl.format(company=comp)
#                     save_data_to_json(data, filename=fn)


In [65]:
company = ['HPG', 'HSG']
for comp in company:
    # Lưu industry_news
    for page in range(1, 170):
        data = industry_news(comp, page=page)
        if not data:
            break
        save_data_to_json(data, filename=f'{comp}_industry_news.json')
    print(f'Đã lưu industry_news cho {comp}')
    # Lưu company_news
    for page in range(1, 25):
        data = company_news(comp, page=page)
        if not data:
            break
        for i in range(1, 6):
            if i == 1:
                save_data_to_json(data[1], filename=f'{comp}_Results_Business_Plan.json')
            if i == 2:
                save_data_to_json(data[2], filename=f'{comp}_Devidend.json')
            if i == 3:
                save_data_to_json(data[3], filename=f'{comp}_Capital_Increase_M&A.json')
            if i == 4:
                save_data_to_json(data[4], filename=f'{comp}_Shareholder_transactions.json')
            if i == 5:
                save_data_to_json(data[5], filename=f'{comp}_Personnel_changes.json')
    print(f'Đã lưu company_news cho {comp}')
    # Lưu events_news
    typeID = [1, 2, 5]
    for id in typeID:
        for page in range(1, 10):
            data = events_news(comp, typeID=id, page=page)
            if not data:
                break

            if id == 1:
                save_data_to_json(data, filename=f'{comp}_event_news.json')
            if id == 2:
                save_data_to_json(data, filename=f'{comp}_listing_news.json')
            if id == 5:
                save_data_to_json(data, filename=f'{comp}_Shareholders_meeting_news.json')
    print(f'Đã lưu events_news cho {comp}')

typeID = [1, 2, 5]
for id in typeID:
    for page in range(1, 200):
        data = same_industry_events_news('HPG', typeID=id, page=page)
        if not data:
            break

        if id == 1:
            save_data_to_json(data, filename='same_industry_event_news.json')
        if id == 2:
            save_data_to_json(data, filename='same_industry_listing_news.json')
        if id == 5:
            save_data_to_json(data, filename='same_industry_Shareholders_meeting_news.json')
print('Đã lưu lịch sự kiện cùng ngành')
            

Đã lưu industry_news cho HPG
Đã lưu company_news cho HPG
Đã lưu events_news cho HPG
Đã lưu industry_news cho HSG
Đã lưu company_news cho HSG
Đã lưu events_news cho HSG
Đã lưu lịch sự kiện cùng ngành


# Craws company

In [ ]:
from vnstock import Listing, Quote, Company, Finance, Trading, Screener
import pandas as pd
from datetime import datetime, date
import time
from hdfs import InsecureClient
import io

In [103]:
client = InsecureClient('http://localhost:9870', user='user')

In [ ]:
def get_all_company(source='vci'):
    listing = Listing(source=source)
    data = listing.all_symbols()
    # data.to_csv(os.path.join(save_dir, 'all_company.csv'), index=False, encoding='utf-8-sig')
    buffer = io.BytesIO()
    data.to_parquet(buffer, index=False)
    buffer.seek(0)

    hdfs_path = '/datalake/raw/all_company.csv'
    client.write(hdfs_path, buffer, overwrite=True)

    print(f"Đã upload lên HDFS tại {hdfs_path}")
    return data
    
get_all_company()

,symbol,organ_name
0,YTC,Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố ...
1,YEG,Công ty Cổ phần Tập đoàn Yeah1
2,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
3,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
4,XPH,Công ty Cổ phần Xà phòng Hà Nội
...,...,...
1711,AAS,Công ty Cổ phần Chứng khoán SmartInvest
1712,AAM,Công ty Cổ Phần Thủy Sản MeKong
1713,AAH,Công ty Cổ phần Hợp Nhất
1714,AAA,Công ty Cổ phần Nhựa An Phát Xanh


In [ ]:
def get_info_company(source='vci'):
    # company = get_all_company()
    # list_company = company['symbol'].to_list()
    list_company = ['HPG', 'HSG', 'DGC', 'GVR', 'VNM', 'FPT', 'MWG', 'SSI', 'CTG', 'BID', 'VGS',
                    'VGL', 'NKG', 'TVN', 'GDA'] 
    data = pd.DataFrame()
    for i, comp in enumerate(list_company, 1):
        try:
            company = Company(source=source, symbol=comp)
            df_ov = company.overview()
            data = pd.concat([data, df_ov], ignore_index=True)
            print(f'[{i}/{len(list_company)}] Đã lấy thông tin công ty {comp}')
        except Exception as e:
            print(f"Lỗi khi lấy thông tin công ty {comp}: {e}")
        
        # Sau mỗi 20 request, sleep 30 giây
        if i % 20 == 0:
            print("Đã cào 20 công ty, chờ 10 giây để tránh rate‑limit…")
            time.sleep(30)

    print("Hoàn tất cào dữ liệu cho tất cả công ty.")
    # data.to_csv(os.path.join(save_dir, 'info_all_company.csv'), index=False, encoding='utf-8-sig')
    buffer = io.BytesIO()
    data.to_parquet(buffer, index=False)
    buffer.seek(0)

    hdfs_path = '/datalake/raw/info_all_company.csv'
    client.write(hdfs_path, buffer, overwrite=True)

    print(f"Đã upload lên HDFS tại {hdfs_path}")
    return data

get_info_company()

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


[1/15] Đã lấy thông tin công ty HPG


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


[2/15] Đã lấy thông tin công ty HSG


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


...

DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.


[12/15] Đã lấy thông tin công ty VGL
[13/15] Đã lấy thông tin công ty NKG


DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.


[14/15] Đã lấy thông tin công ty TVN
[15/15] Đã lấy thông tin công ty GDA
Hoàn tất cào dữ liệu cho tất cả công ty.
Đã upload lên HDFS tại /datalake/raw/info_all_company.parquet


,symbol,id,issue_share,history,company_profile,icb_name3,icb_name2,icb_name4,financial_ratio_issue_share,charter_capital
0,HPG,70880,6396250200,- Năm 2000: Công ty TNHH Sắt thép Hòa Phát đư...,Công ty Cổ phần Tập đoàn Hoà Phát (HPG) là một...,Kim loại,Tài nguyên Cơ bản,Thép và sản phẩm thép,7675465855,63962502000000
1,HSG,70975,620982309,- Ngày 08/08/2001: Tiền thân là Công ty Cổ ph...,"Công ty Cổ phần Tập đoàn Hoa Sen (HSG), có tiề...",Kim loại,Tài nguyên Cơ bản,Thép và sản phẩm thép,620982309,6209823090000
2,DGC,69260,379779286,- 1963: Công ty Hóa chất Đức Giang được thành...,Công ty Cổ phần Tập đoàn Hóa chất Đức Giang (D...,Hóa chất,Hóa chất,"Sản phẩm hóa dầu, Nông dược & Hóa chất khác",379778413,3797792860000
3,GVR,76558,4000000000,- Ngày 29/04/1995: Tổng Công ty Cao su Việt N...,Tập đoàn Công nghiệp Cao su Việt Nam (GVR) có ...,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",4000000000,40000000000000
4,VNM,76529,2089955445,- Năm 1976: Tiền thân của công ty là Công ty ...,Công ty Cổ phần Sữa Việt Nam (VNM) có tiền thâ...,Sản xuất thực phẩm,Thực phẩm và đồ uống,Thực phẩm,2089955445,20899554450000
5,FPT,69998,1481330122,- Ngày 13/09/1988: Tiền thân của Công ty FPT ...,Công ty Cổ phần FPT (FPT) có tiền thân là Công...,Phần mềm & Dịch vụ Máy tính,Công nghệ Thông tin,Phần mềm,1481330122,14813301220000
6,MWG,72296,1479693177,- Tháng 03/2004: Công ty TNHH Thế Giới Di Độn...,Công ty Cổ phần Đầu tư Thế Giới Di Động (MWG) ...,Bán lẻ,Bán lẻ,Phân phối hàng chuyên dụng,1478609048,14796931770000
7,SSI,74416,1973863918,- Năm 1999: Công ty Cổ phần Chứng khoán Sài G...,Công ty Cổ phần Chứng khoán SSI (SSI) có tiền ...,Dịch vụ tài chính,Dịch vụ tài chính,Môi giới chứng khoán,1971872450,19738639180000
8,CTG,68964,5369991748,- 1988: Ngân hàng Công thương Việt Nam được t...,Ngân hàng Thương mại Cổ phần Công thương Việt ...,Ngân hàng,Ngân hàng,Ngân hàng,5369991748,53699917480000
9,BID,68105,7021361917,- Ngày 26/04/1957: Tiền thân là Ngân hàng Kiế...,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,Ngân hàng,Ngân hàng,Ngân hàng,7021361917,70213619170000


In [ ]:
def get_stock_price(source='vci'):
    # company = get_all_company()
    # list_company = company['symbol'].to_list()
    list_company = ['HPG', 'HSG', 'DGC', 'GVR', 'VNM', 'FPT', 'MWG', 'SSI', 'CTG', 'BID', 'VGS',
                    'VGL', 'NKG', 'TVN', 'GDA'] 
    data = pd.DataFrame()
    for i, comp in enumerate(list_company, 1):
        try:
            quote = Quote(source=source, symbol=comp)
            his_price = quote.history(start='2015-01-01', end='2025-07-14')
            if not his_price.empty:
                his_price['symbol'] = comp
                data = pd.concat([data, his_price], ignore_index=True)
                print(f'[{i}/{len(list_company)}] Đã lấy dữ liệu giá cổ phiếu cho {comp}')
        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu giá cổ phiếu cho {comp}: {e}")
        
    print("Hoàn tất cào dữ liệu cho tất cả công ty.")
    # data.to_csv(os.path.join(save_dir, 'stock_price_all_company.csv'), index=False, encoding='utf-8-sig')
    buffer = io.BytesIO()
    data.to_parquet(buffer, index=False)
    buffer.seek(0)

    hdfs_path = '/datalake/raw/stock_price_all_company.csv'
    client.write(hdfs_path, buffer, overwrite=True)

    print(f"Đã upload lên HDFS tại {hdfs_path}")
    return data

get_stock_price()

[1/15] Đã lấy dữ liệu giá cổ phiếu cho HPG
[2/15] Đã lấy dữ liệu giá cổ phiếu cho HSG
[3/15] Đã lấy dữ liệu giá cổ phiếu cho DGC
[4/15] Đã lấy dữ liệu giá cổ phiếu cho GVR
[5/15] Đã lấy dữ liệu giá cổ phiếu cho VNM
[6/15] Đã lấy dữ liệu giá cổ phiếu cho FPT
[7/15] Đã lấy dữ liệu giá cổ phiếu cho MWG
[8/15] Đã lấy dữ liệu giá cổ phiếu cho SSI
[9/15] Đã lấy dữ liệu giá cổ phiếu cho CTG
[10/15] Đã lấy dữ liệu giá cổ phiếu cho BID
[11/15] Đã lấy dữ liệu giá cổ phiếu cho VGS
[12/15] Đã lấy dữ liệu giá cổ phiếu cho VGL
[13/15] Đã lấy dữ liệu giá cổ phiếu cho NKG
[14/15] Đã lấy dữ liệu giá cổ phiếu cho TVN
[15/15] Đã lấy dữ liệu giá cổ phiếu cho GDA
Hoàn tất cào dữ liệu cho tất cả công ty.
Đã upload lên HDFS tại /datalake/raw/stock_price_all_company.parquet


,time,open,high,low,close,volume,symbol
0,2015-01-05,3.44,3.51,3.41,3.44,207020,HPG
1,2015-01-06,3.38,3.47,3.38,3.47,260380,HPG
2,2015-01-07,3.47,3.51,3.44,3.44,232770,HPG
3,2015-01-08,3.44,3.47,3.41,3.44,107380,HPG
4,2015-01-09,3.44,3.51,3.41,3.51,409230,HPG
...,...,...,...,...,...,...,...
35761,2025-07-08,19.80,20.40,19.60,20.10,205600,GDA
35762,2025-07-09,20.00,20.30,19.90,20.00,76100,GDA
35763,2025-07-10,20.00,20.00,19.60,19.70,156800,GDA
35764,2025-07-11,19.90,19.90,19.40,19.70,156600,GDA


In [ ]:
def get_ratio_summary(source='vci'):
    # company = get_all_company()
    # list_company = company['symbol'].to_list()
    list_company = ['HPG', 'HSG', 'DGC', 'GVR', 'VNM', 'FPT', 'MWG', 'SSI', 'CTG', 'BID', 'VGS',
                    'VGL', 'NKG', 'TVN', 'GDA'] 
    data = pd.DataFrame()
    for i, comp in enumerate(list_company, 1):
        try:
            comp_rs = Company(source=source, symbol=comp)
            rs = comp_rs.ratio_summary()
            if not rs.empty:
                data = pd.concat([data, rs], ignore_index=True)
                print(f'[{i}/{len(list_company)}] Đã lấy dữ liệu ratio summay cho {comp}')
        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu ratio summary cho {comp}: {e}")
        
    print("Hoàn tất cào dữ liệu cho tất cả công ty.")
    # data.to_csv(os.path.join(save_dir, 'ratio_summary_all_company.csv'), index=False, encoding='utf-8-sig')
    buffer = io.BytesIO()
    data.to_parquet(buffer, index=False)
    buffer.seek(0)

    hdfs_path = '/datalake/raw/ratio_summary_all_company.csv'
    client.write(hdfs_path, buffer, overwrite=True)

    print(f"Đã upload lên HDFS tại {hdfs_path}")
    return data

get_ratio_summary()

[1/15] Đã lấy dữ liệu ratio summay cho HPG
[2/15] Đã lấy dữ liệu ratio summay cho HSG
[3/15] Đã lấy dữ liệu ratio summay cho DGC
[4/15] Đã lấy dữ liệu ratio summay cho GVR
[5/15] Đã lấy dữ liệu ratio summay cho VNM
[6/15] Đã lấy dữ liệu ratio summay cho FPT
[7/15] Đã lấy dữ liệu ratio summay cho MWG


The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


[8/15] Đã lấy dữ liệu ratio summay cho SSI
[9/15] Đã lấy dữ liệu ratio summay cho CTG


The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


[10/15] Đã lấy dữ liệu ratio summay cho BID
[11/15] Đã lấy dữ liệu ratio summay cho VGS


The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


[12/15] Đã lấy dữ liệu ratio summay cho VGL
[13/15] Đã lấy dữ liệu ratio summay cho NKG
[14/15] Đã lấy dữ liệu ratio summay cho TVN
[15/15] Đã lấy dữ liệu ratio summay cho GDA
Hoàn tất cào dữ liệu cho tất cả công ty.
Đã upload lên HDFS tại /datalake/raw/ratio_summary_all_company.parquet


,symbol,year_report,length_report,update_date,revenue,revenue_growth,net_profit,net_profit_growth,ebit_margin,roe,...,rtq4,charter_capital_ratio,rtq10,dividend,ebitda,ebit,le,de,ccc,rtq17
0,HPG,2025,1,1745945062950,37950635502050,0.220569,3344284694388,0.165017,0.115335,0.110522,...,0.230997,1.844782,0.939556,0.077670,22752951358569,4339094411460,0.985606,0.939556,192.745495,2.895898
1,HSG,2025,1,1745842812770,8663002568692,-0.080718,205426811430,-0.356466,0.019844,0.041721,...,0.000000,1.821484,0.551368,0.000000,1413537721245,167715201516,0.312291,0.551368,137.507975,3.636218
2,DGC,2025,1,1745255665633,2811403662097,0.178383,809334908466,0.203266,0.294814,0.230773,...,0.000000,3.769188,0.153796,0.028874,3376367149752,828519347208,0.075421,0.153796,80.427516,7.231670
3,GVR,2025,1,1746435600583,5681992269243,0.237817,1184647986486,1.491044,0.197711,0.089113,...,0.043358,1.502080,0.385409,0.013051,7359538862120,1122399120258,0.157084,0.385409,105.302653,5.080244
4,VNM,2025,1,1745949008247,12965404856436,-0.082088,1568482188101,-0.285321,0.124457,0.266259,...,0.004092,1.800157,0.462258,0.038408,11696343274578,1609795815116,0.275579,0.462258,131.711316,5.619316
5,FPT,2025,1,1745412996577,16064980391264,0.139902,2174301386525,0.209268,0.161608,0.276631,...,0.026062,2.558286,0.952618,0.015736,13347374679962,2595124918148,0.535550,0.952618,110.578368,19.102382
6,MWG,2025,1,1745830800630,36332913479242,0.146691,1545937090941,0.713069,0.043803,0.158598,...,0.000000,2.005087,1.449658,0.000000,7346700380533,1582804484839,0.883820,1.449658,107.493561,4.964138
7,SSI,2025,1,1745950756570,2159272711993,0.094509,843994524895,0.104342,0.685754,0.113302,...,0.000000,1.403509,2.033709,0.000000,5230503115369,1480730817587,1.994229,2.033709,NaN,0.000000
8,CTG,2025,1,1745948663757,32684212000000,0.055966,5418875000000,0.092777,0.000000,0.178158,...,0.000000,2.867456,15.039931,0.000000,0,0,0.000000,15.039931,NaN,0.000000
9,BID,2025,1,1745835790780,35286235000000,0.050311,5840373000000,0.004770,0.000000,0.175393,...,0.000000,2.220457,17.319406,0.000000,0,0,0.000000,17.319406,NaN,0.000000


In [ ]:
def get_balance_sheet_by_quarter(source='vci'):
    # company = get_all_company()
    # list_company = company['symbol'].to_list()
    list_company = ['HPG', 'HSG', 'DGC', 'GVR', 'VNM', 'FPT', 'MWG', 'SSI', 'CTG', 'BID', 'VGS',
                    'VGL', 'NKG', 'TVN', 'GDA'] 
    data = pd.DataFrame()
    for i, comp in enumerate(list_company, 1):
        try:
            comp_b = Finance(source=source, symbol=comp)
            bs = comp_b.balance_sheet(lang='vi')
            if not bs.empty:
                data = pd.concat([data, bs], ignore_index=True)
                print(f'[{i}/{len(list_company)}] Đã lấy dữ liệu balance sheet cho {comp}')
        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu balance sheet cho {comp}: {e}")
        
    print("Hoàn tất cào dữ liệu cho tất cả công ty.")
    # data.to_csv(os.path.join(save_dir, 'balance_sheet_all_company.csv'), index=False, encoding='utf-8-sig')
    buffer = io.BytesIO()
    data.to_parquet(buffer, index=False)
    buffer.seek(0)

    hdfs_path = '/datalake/raw/balance_sheet_all_company.csv'
    client.write(hdfs_path, buffer, overwrite=True)

    print(f"Đã upload lên HDFS tại {hdfs_path}")
    return data

get_balance_sheet_by_quarter()

[1/15] Đã lấy dữ liệu balance sheet cho HPG
[2/15] Đã lấy dữ liệu balance sheet cho HSG
[3/15] Đã lấy dữ liệu balance sheet cho DGC
[4/15] Đã lấy dữ liệu balance sheet cho GVR
[5/15] Đã lấy dữ liệu balance sheet cho VNM
[6/15] Đã lấy dữ liệu balance sheet cho FPT
[7/15] Đã lấy dữ liệu balance sheet cho MWG
[8/15] Đã lấy dữ liệu balance sheet cho SSI
[9/15] Đã lấy dữ liệu balance sheet cho CTG
[10/15] Đã lấy dữ liệu balance sheet cho BID
[11/15] Đã lấy dữ liệu balance sheet cho VGS
[12/15] Đã lấy dữ liệu balance sheet cho VGL
[13/15] Đã lấy dữ liệu balance sheet cho NKG
[14/15] Đã lấy dữ liệu balance sheet cho TVN
[15/15] Đã lấy dữ liệu balance sheet cho GDA
Hoàn tất cào dữ liệu cho tất cả công ty.
Đã upload lên HDFS tại /datalake/raw/balance_sheet_all_company.parquet


,CP,Năm,Kỳ,TÀI SẢN NGẮN HẠN (đồng),Tiền và tương đương tiền (đồng),Giá trị thuần đầu tư ngắn hạn (đồng),Các khoản phải thu ngắn hạn (đồng),Hàng tồn kho ròng,Tài sản lưu động khác,TÀI SẢN DÀI HẠN (đồng),...,_Các công cụ tài chính phái sinh và khoản nợ tài chính khác,"Vốn tài trợ, uỷ thác đầu tư của CP và các tổ chức TD khác",Phát hành giấy tờ có giá,Các khoản nợ khác,Vốn của tổ chức tín dụng,Quỹ của tổ chức tín dụng,Chênh lệch tỷ giá hối đoái,Đầu tư vào công ty con,Tài sản cố định thuê tài chính,Chênh lệch đánh giá lại tài sản
0,HPG,2025,1,8.891420e+13,7459035818129,1.621251e+13,1.085605e+13,4.660384e+13,7.782752e+12,1.399474e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HPG,2024,4,8.667428e+13,6887646139852,1.897472e+13,7.647800e+12,4.609122e+13,7.072891e+12,1.378154e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HPG,2024,3,8.084180e+13,8500998079280,1.638680e+13,9.140948e+12,4.019782e+13,6.615233e+12,1.305445e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HPG,2024,2,8.446683e+13,13032037853283,1.531467e+13,1.039280e+13,4.016354e+13,5.563778e+12,1.221422e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HPG,2024,1,9.403246e+13,12428888036237,2.227142e+13,1.210512e+13,4.271428e+13,4.512755e+12,1.079078e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,GDA,2023,1,8.478172e+12,350080009996,1.939000e+12,2.125082e+12,3.835363e+12,2.286475e+11,2.797526e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,GDA,2022,4,7.402067e+12,829814067681,1.162000e+12,1.977812e+12,3.269415e+12,1.630262e+11,2.896046e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,GDA,2022,3,8.975541e+12,743317191937,1.214000e+12,2.489225e+12,4.390275e+12,1.387234e+11,2.947664e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,GDA,2022,2,1.191387e+13,358517999542,1.998767e+12,3.398787e+12,5.663673e+12,4.941298e+11,3.051470e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_income_statement_by_quarter(source='vci'):
    # company = get_all_company()
    # list_company = company['symbol'].to_list()
    list_company = ['HPG', 'HSG', 'DGC', 'GVR', 'VNM', 'FPT', 'MWG', 'SSI', 'CTG', 'BID', 'VGS',
                    'VGL', 'NKG', 'TVN', 'GDA'] 
    data = pd.DataFrame()
    for i, comp in enumerate(list_company, 1):
        try:
            comp_b = Finance(source=source, symbol=comp)
            ins = comp_b.income_statement(lang='vi')
            if not ins.empty:
                data = pd.concat([data, ins], ignore_index=True)
                print(f'[{i}/{len(list_company)}] Đã lấy dữ liệu income statement cho {comp}')
        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu income statement cho {comp}: {e}")
        
    print("Hoàn tất cào dữ liệu cho tất cả công ty.")
    # data.to_csv(os.path.join(save_dir, 'income_statement_all_company.csv'), index=False, encoding='utf-8-sig')
    buffer = io.BytesIO()
    data.to_parquet(buffer, index=False)
    buffer.seek(0)

    hdfs_path = '/datalake/raw/income_statement_all_company.csv'
    client.write(hdfs_path, buffer, overwrite=True)

    print(f"Đã upload lên HDFS tại {hdfs_path}")
    return data

get_income_statement_by_quarter()

[1/15] Đã lấy dữ liệu income statement cho HPG
[2/15] Đã lấy dữ liệu income statement cho HSG
[3/15] Đã lấy dữ liệu income statement cho DGC
[4/15] Đã lấy dữ liệu income statement cho GVR
[5/15] Đã lấy dữ liệu income statement cho VNM
[6/15] Đã lấy dữ liệu income statement cho FPT
[7/15] Đã lấy dữ liệu income statement cho MWG
[8/15] Đã lấy dữ liệu income statement cho SSI
[9/15] Đã lấy dữ liệu income statement cho CTG
[10/15] Đã lấy dữ liệu income statement cho BID
[11/15] Đã lấy dữ liệu income statement cho VGS
[12/15] Đã lấy dữ liệu income statement cho VGL
[13/15] Đã lấy dữ liệu income statement cho NKG
[14/15] Đã lấy dữ liệu income statement cho TVN
[15/15] Đã lấy dữ liệu income statement cho GDA
Hoàn tất cào dữ liệu cho tất cả công ty.
Đã upload lên HDFS tại /datalake/raw/income_statement_all_company.parquet


,CP,Năm,Kỳ,Tăng trưởng doanh thu (%),Doanh thu (đồng),Lợi nhuận sau thuế của Cổ đông công ty mẹ (đồng),Tăng trưởng lợi nhuận (%),Thu nhập tài chính,Chi phí tiền lãi vay,Doanh thu bán hàng và cung cấp dịch vụ,...,Chứng khoán đầu tư,Hoạt động khác,Chi phí hoạt động khác,Lãi/lỗ thuần từ hoạt động khác,Cố tức đã nhận,Tổng thu nhập hoạt động,LN từ HĐKD trước CF dự phòng,Chi phí dự phòng rủi ro tín dụng,Thuế TNDN,Lãi cơ bản trên cổ phiếu
0,HPG,2025,1,0.220569,37950635502050,3344284694388,0.165017,4.380572e+11,-6.270244e+11,3.795064e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HPG,2024,4,0.008808,35232197602514,2808645369171,-0.055212,7.005601e+11,-5.624927e+11,3.523220e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HPG,2024,3,0.192404,34300352057045,3022946632050,0.507896,5.284532e+11,-5.251148e+11,3.430035e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HPG,2024,2,0.340166,39936268160790,3319263141862,1.273402,6.450601e+11,-5.641344e+11,3.993627e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HPG,2024,1,0.157344,31092569628223,2870588692991,6.221656,7.519554e+11,-6.356190e+11,3.109257e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,GDA,2023,1,-0.378198,3941970124297,81621644134,-0.600988,7.478570e+10,-8.282249e+10,3.941970e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,GDA,2022,4,0.000000,4358502638165,-399200061449,0.000000,9.515713e+10,-6.436965e+10,4.358503e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,GDA,2022,3,0.000000,4216261172124,-206777944101,0.000000,1.180754e+11,-6.909426e+10,4.216261e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,GDA,2022,2,0.000000,6766475351009,125866607350,0.000000,1.009466e+11,-6.236934e+10,6.766475e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_cash_flow_by_quarter(source='vci'):
    # company = get_all_company()
    # list_company = company['symbol'].to_list()
    list_company = ['HPG', 'HSG', 'DGC', 'GVR', 'VNM', 'FPT', 'MWG', 'SSI', 'CTG', 'BID', 'VGS',
                    'VGL', 'NKG', 'TVN', 'GDA'] 
    data = pd.DataFrame()
    for i, comp in enumerate(list_company, 1):
        try:
            comp_cf = Finance(source=source, symbol=comp)
            cf = comp_cf.cash_flow(lang='vi')
            if not cf.empty:
                data = pd.concat([data, cf], ignore_index=True)
                print(f'[{i}/{len(list_company)}] Đã lấy dữ liệu cash flow cho {comp}')
        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu cash flow cho {comp}: {e}")
        
    print("Hoàn tất cào dữ liệu cho tất cả công ty.")
    # data.to_csv(os.path.join(save_dir, 'cash_flow_all_company.csv'), index=False, encoding='utf-8-sig')
    buffer = io.BytesIO()
    data.to_parquet(buffer, index=False)
    buffer.seek(0)

    hdfs_path = '/datalake/raw/cash_flow_all_company.csv'
    client.write(hdfs_path, buffer, overwrite=True)

    print(f"Đã upload lên HDFS tại {hdfs_path}")
    return data

get_cash_flow_by_quarter()

[1/15] Đã lấy dữ liệu cash flow cho HPG
[2/15] Đã lấy dữ liệu cash flow cho HSG
[3/15] Đã lấy dữ liệu cash flow cho DGC
[4/15] Đã lấy dữ liệu cash flow cho GVR
[5/15] Đã lấy dữ liệu cash flow cho VNM
[6/15] Đã lấy dữ liệu cash flow cho FPT
[7/15] Đã lấy dữ liệu cash flow cho MWG
[8/15] Đã lấy dữ liệu cash flow cho SSI
[9/15] Đã lấy dữ liệu cash flow cho CTG
[10/15] Đã lấy dữ liệu cash flow cho BID
[11/15] Đã lấy dữ liệu cash flow cho VGS
[12/15] Đã lấy dữ liệu cash flow cho VGL
[13/15] Đã lấy dữ liệu cash flow cho NKG
[14/15] Đã lấy dữ liệu cash flow cho TVN
[15/15] Đã lấy dữ liệu cash flow cho GDA
Hoàn tất cào dữ liệu cho tất cả công ty.
Đã upload lên HDFS tại /datalake/raw/cash_flow_all_company.parquet


,CP,Năm,Kỳ,Lãi/Lỗ ròng trước thuế,Khấu hao TSCĐ,Dự phòng RR tín dụng,Lãi/Lỗ chênh lệch tỷ giá chưa thực hiện,Lãi/Lỗ từ hoạt động đầu tư,Thu nhập lãi,Thu lãi và cổ tức,...,Tiền và tương đương tiền,Ảnh hưởng của chênh lệch tỷ giá,Tiền và tương đương tiền cuối kỳ,Lãi/Lỗ từ thanh lý tài sản cố định,Cố tức đã nhận,_Tăng/Giảm các khoản phải thu,_Tăng/Giảm các khoản phải trả,(Lãi)/lỗ các hoạt động khác,Lưu chuyển tiền thuần từ HĐKD trước thuế,Chi từ các quỹ của TCTD
0,HPG,2025,1,3.839765e+12,1.801603e+12,1.128911e+10,1.326165e+11,-2.447268e+11,6.270244e+11,0.0,...,6.887646e+12,-4.527786e+08,7.459036e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HPG,2024,4,3.286676e+12,1.693028e+12,-6.103512e+10,2.277512e+11,-7.653708e+11,5.624927e+11,0.0,...,8.500998e+12,5.563162e+09,6.887646e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HPG,2024,3,3.412385e+12,1.771186e+12,-3.156807e+09,-2.067842e+11,1.030117e+11,5.251148e+11,0.0,...,1.303204e+13,-7.841987e+09,8.500998e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HPG,2024,2,3.733226e+12,1.729641e+12,1.620225e+11,-3.029061e+10,4.063399e+11,5.641344e+11,0.0,...,1.242889e+13,1.165316e+09,1.303204e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HPG,2024,1,3.261215e+12,1.721816e+12,1.066982e+10,1.827942e+11,-1.462291e+12,6.356190e+11,0.0,...,1.225200e+13,1.878498e+09,1.242889e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,GDA,2023,1,8.664943e+10,8.522400e+10,-3.693204e+10,0.000000e+00,-4.136307e+10,8.282249e+10,NaN,...,8.298141e+11,0.000000e+00,3.500800e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,GDA,2022,4,-4.554763e+11,8.571561e+10,3.873299e+09,-1.718306e+09,-5.189414e+10,6.436965e+10,NaN,...,7.433172e+11,-2.956751e+09,8.298141e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,GDA,2022,3,-2.519506e+11,8.656312e+10,-3.729083e+09,-8.332041e+09,-7.933547e+10,6.909426e+10,NaN,...,3.585180e+11,-9.927396e+07,7.433172e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,GDA,2022,2,1.583458e+11,8.615778e+10,1.054326e+11,8.332041e+09,-5.052472e+10,6.236934e+10,NaN,...,4.682056e+11,9.927396e+07,3.585180e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
def get_ratio_by_quarter(source='vci'):
    # company = get_all_company()
    # list_company = company['symbol'].to_list()
    list_company = ['HPG', 'HSG', 'DGC', 'GVR', 'VNM', 'FPT', 'MWG', 'SSI', 'CTG', 'BID', 'VGS',
                    'VGL', 'NKG', 'TVN', 'GDA'] 
    data = pd.DataFrame()
    for i, comp in enumerate(list_company, 1):
        try:
            comp_r = Finance(source=source, symbol=comp)
            r = comp_r.ratio(lang='vi')
            if not r.empty:
                data = pd.concat([data, r], ignore_index=True)
                print(f'[{i}/{len(list_company)}] Đã lấy dữ liệu ratio cho {comp}')
        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu ratio cho {comp}: {e}")
        
    print("Hoàn tất cào dữ liệu cho tất cả công ty.")
    # data.to_csv(os.path.join(save_dir, 'ratio_all_company.csv'), index=False, encoding='utf-8-sig')
    buffer = io.BytesIO()
    data.to_csv(buffer, index=False)
    buffer.seek(0)

    hdfs_path = '/datalake/raw/ratio_all_company.csv'
    client.write(hdfs_path, buffer, overwrite=True)

    print(f"Đã upload lên HDFS tại {hdfs_path}")
    return data

get_ratio_by_quarter()

[1/15] Đã lấy dữ liệu ratio cho HPG
[2/15] Đã lấy dữ liệu ratio cho HSG
[3/15] Đã lấy dữ liệu ratio cho DGC
[4/15] Đã lấy dữ liệu ratio cho GVR
[5/15] Đã lấy dữ liệu ratio cho VNM
[6/15] Đã lấy dữ liệu ratio cho FPT
[7/15] Đã lấy dữ liệu ratio cho MWG
[8/15] Đã lấy dữ liệu ratio cho SSI


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[9/15] Đã lấy dữ liệu ratio cho CTG


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[10/15] Đã lấy dữ liệu ratio cho BID
[11/15] Đã lấy dữ liệu ratio cho VGS
[12/15] Đã lấy dữ liệu ratio cho VGL
[13/15] Đã lấy dữ liệu ratio cho NKG
[14/15] Đã lấy dữ liệu ratio cho TVN
[15/15] Đã lấy dữ liệu ratio cho GDA
Hoàn tất cào dữ liệu cho tất cả công ty.
Đã upload lên HDFS tại /datalake/raw/ratio_all_company.csv


Meta          Chỉ tiêu cơ cấu nguồn vốn                           \
      CP   Năm Kỳ          (Vay NH+DH)/VCSH   Nợ/VCSH TSCĐ / Vốn CSH   
0    HPG  2025  1                  0.985606  0.939556       0.727461   
1    HPG  2024  4                  0.959843  0.958087       0.588137   
2    HPG  2024  3                  0.923392  0.891108       0.612598   
3    HPG  2024  2                  0.852670  0.901137       0.644285   
4    HPG  2024  1                  0.886144  0.911868       0.672344   
..   ...   ... ..                       ...       ...            ...   
624  GDA  2023  1                  1.715177  2.194384       0.696067   
625  GDA  2022  4                  1.547320  1.988379       0.736662   
626  GDA  2022  3                  0.000000  0.000000       0.678135   
627  GDA  2022  2                  0.000000  0.000000       0.660150   
628  GDA  2022  1                  0.000000  0.000000       0.679116   

                        Chỉ tiêu hiệu quả hoạt động                 \
    Vốn CSH/Vốn điều lệ           Vòng quay tài sản Vòng quay TSCĐ   
0              1.844782                    0.668503       1.996491   
1              1.792417                    0.673609       1.991800   
2              1.747572                    0.687111       1.971378   
3              1.699067                    0.692487       1.865096   
4              1.651352                    0.666510       1.730895   
..                  ...                         ...            ...   
624            3.077691                    1.588017       7.482421   
625            3.007631                    1.883315       8.041710   
626            3.355696                    0.000000       0.000000   
627            3.536022                    0.000000       0.000000   
628            3.530825                    0.000000       0.000000   

                                ... Chỉ tiêu thanh khoản Chỉ tiêu định giá  \
    Số ngày thu tiền bình quân  ...    Đòn bẩy tài chính Vốn hóa (Tỷ đồng)   
0                    14.720558  ...             1.939556      1.976432e+14   
1                    13.605410  ...             1.958087      1.653431e+14   
2                    16.210015  ...             1.891108      1.653431e+14   
3                    16.215280  ...             1.901137      1.730186e+14   
4                    16.295421  ...             1.911868      1.822931e+14   
..                         ...  ...                  ...               ...   
624                  32.197062  ...             3.194384      0.000000e+00   
625                  30.007802  ...             2.985395      0.000000e+00   
626                   0.000000  ...             3.097984      0.000000e+00   
627                   0.000000  ...             3.690120      0.000000e+00   
628                   0.000000  ...             3.802655      0.000000e+00   

                                                                          \
    Số CP lưu hành (Triệu CP)        P/E       P/B       P/S P/Cash Flow   
0                7.675466e+09  15.817610  1.679199  1.357212   64.133449   
1                6.396250e+09  13.754011  1.445857  1.190760   25.020437   
2                6.396250e+09  13.568751  1.481703  1.191679   18.528919   
3                6.396250e+09  15.493216  1.593555  1.298200   18.620329   
4                6.396250e+09  19.584237  1.727562  1.479452   16.099007   
..                        ...        ...       ...       ...         ...   
624              1.146915e+08   0.000000  0.000000  0.000000    0.000000   
625              1.146915e+08   0.000000  0.000000  0.000000    0.000000   
626              1.146915e+08   0.000000  0.000000  0.000000    0.000000   
627              1.146915e+08   0.000000  0.000000  0.000000    0.000000   
628              1.146915e+08   0.000000  0.000000  0.000000    0.000000   

                                           
       EPS (VND)    BVPS (VND)  EV/EBITDA  
0     435.710973  15334.695560  12.285072  
1     439.108115  17

In [ ]:
# quote = Quote(symbol='HPG')
# data = quote.history(start='2015-01-01', end='2025-07-14')
# data.to_csv(r"C:\Users\user\Documents\python\Samsung\Data\HPG_price.csv", index=False)

In [ ]:
# quote = Quote(symbol='HSG')
# data = quote.history(start='2015-01-01', end='2025-07-14')
# data.to_csv(r"C:\Users\user\Documents\python\Samsung\Data\HSG_price.csv", index=False)